In [1]:
import streamlit as st
st.title('Librosa la sabrosa')

2024-05-21 19:48:15.929 
  command:

    streamlit run C:\Users\src4\AppData\Roaming\Python\Python311\site-packages\ipykernel_launcher.py [ARGUMENTS]


DeltaGenerator()